In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Import data

In [8]:
#btc options data
df=pd.read_csv('data/btc_data.csv', sep=",", index_col=0)
df = df.reset_index(drop=False)
#df.drop(columns=['Unnamed: 0', 'interest_rate'], inplace=True)

In [12]:
import pandas as pd
import QuantLib as ql

# Load the BTC options data
btc_data_new = df  # Replace with the actual path

In [28]:

# Extract relevant data for the Heston model application
spot_price = btc_data_new['underlying_price'].iloc[0]  # Underlying BTC price
strike_price = float(btc_data_new['Strike Price'].iloc[0])  # Ensure strike price is a float
maturity_date = ql.Date(4, 2, 2024)  # Option expiration date
risk_free_rate = 0.01  # Risk-free interest rate (assumed)
dividend_yield = 0.0  # Dividend yield (assumed)
implied_volatility = btc_data_new['mark_iv'].iloc[0] / 100  # Implied volatility from mark_iv

# Check and adjust if necessary
if implied_volatility <= 0:
    implied_volatility = 0.50  # Use a typical value if the implied volatility is non-positive

# Heston model parameters (example values, these can be tuned)
v0 = implied_volatility**2  # Initial variance from implied volatility
kappa = 2.0  # Mean reversion speed (adjusted)
theta = v0  # Long-term variance set to initial variance
sigma = 0.5  # Volatility of volatility (adjusted)
rho = -0.3  # Correlation (adjusted)

# Set up the Heston process
day_count = ql.Actual365Fixed()
calendar = ql.NullCalendar()
spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(0, calendar, ql.QuoteHandle(ql.SimpleQuote(risk_free_rate)), day_count))
dividend_ts = ql.YieldTermStructureHandle(ql.FlatForward(0, calendar, ql.QuoteHandle(ql.SimpleQuote(dividend_yield)), day_count))
heston_process = ql.HestonProcess(flat_ts, dividend_ts, spot_handle, v0, kappa, theta, sigma, rho)

# Set up the Heston model
heston_model = ql.HestonModel(heston_process)
engine = ql.AnalyticHestonEngine(heston_model)  # Using the semi-analytic engine for Heston model

# Define the European call option
payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
option = ql.VanillaOption(payoff, exercise)
option.setPricingEngine(engine)

# Calculate the option price
option_price = option.NPV()
print("The option price is:", option_price)

# Debugging information
print(f"Spot Price: {spot_price}")
print(f"Strike Price: {strike_price}")
print(f"Implied Volatility: {implied_volatility}")
print(f"Initial Variance (v0): {v0}")
print(f"Risk-Free Rate: {risk_free_rate}")
print(f"Maturity Date: {maturity_date}")
print(f"Day Count: {day_count}")
print(f"Dividend Yield: {dividend_yield}")
print(f"Heston Parameters: kappa={kappa}, theta={theta}, sigma={sigma}, rho={rho}")

The option price is: 0.0
Spot Price: 43046.8715
Strike Price: 38500.0
Implied Volatility: 0.8549
Initial Variance (v0): 0.7308540099999999
Risk-Free Rate: 0.01
Maturity Date: February 4th, 2024
Day Count: Actual/365 (Fixed) day counter
Dividend Yield: 0.0
Heston Parameters: kappa=2.0, theta=0.7308540099999999, sigma=0.5, rho=-0.3


In [ ]:
import pandas as pd
import QuantLib as ql

# Load the BTC options data
btc_data_new = df  # Replace with the actual path

In [ ]:
The option price is: 0.0

In [5]:
import QuantLib as ql
import pandas as pd

# Load the BTC options data
btc_data_new = df

# Extract relevant data from the CSV
spot_price = float(btc_data_new['underlying_price'].iloc[0])  # Ensure the spot price is a float
strike_price = float(btc_data_new['Strike Price'].iloc[0])    # Ensure the strike price is a float
maturity_date_str = btc_data_new['Expiration Date'].iloc[0]

# Convert expiration date string to QuantLib Date object
expiration_date_parts = maturity_date_str.split('FEB')
expiration_day = int(expiration_date_parts[0])
expiration_month = 2  # Assuming February for 'FEB'
expiration_year = 2024  # Assuming the year 2024 for demonstration
maturity_date = ql.Date(expiration_day, expiration_month, expiration_year)

# Extract risk-free rate and convert to decimal
risk_free_rate = float(btc_data_new['interest_rate'].iloc[0]) / 100
dividend_yield = 0.0  # Assuming no dividend yield

# Heston model parameters (assuming some typical values for demonstration)
v0 = 0.04       # Initial variance
kappa = 1.0     # Mean reversion speed
theta = 0.04    # Long-term variance
sigma = 0.2     # Volatility of volatility
rho = -0.5      # Correlation

# Set up the Heston process
day_count = ql.Actual365Fixed()
calendar = ql.NullCalendar()
spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(0, calendar, ql.QuoteHandle(ql.SimpleQuote(risk_free_rate)), day_count))
dividend_ts = ql.YieldTermStructureHandle(ql.FlatForward(0, calendar, ql.QuoteHandle(ql.SimpleQuote(dividend_yield)), day_count))
heston_process = ql.HestonProcess(flat_ts, dividend_ts, spot_handle, v0, kappa, theta, sigma, rho)

# Set up the Heston model
heston_model = ql.HestonModel(heston_process)
engine = ql.AnalyticHestonEngine(heston_model)

# Define the European call option
payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
option = ql.VanillaOption(payoff, exercise)
option.setPricingEngine(engine)

# Calculate the option price
option_price = option.NPV()
print(f"The price of the European call option is: {option_price}")


The price of the European call option is: 0.0
